In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

len4 blp

In [17]:
count=0
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# len4_ndb is left joined so delete duplicated rows
data=data.drop_duplicates(subset=["id","year","総計"])
# drop never treated group
data=data.dropna(subset=["release_year"])
data=data.dropna(subset=["総計"])
data.head()
data.shape

(25798, 21)

In [18]:
duplicate_rows = data[data.duplicated(subset=['year', 'id', '総計'])]

# 重複行の数を表示
print("重複行の数:", len(duplicate_rows))

# 重複行のデータを表示
print(duplicate_rows)

重複行の数: 0
Empty DataFrame
Columns: [薬効分類, 薬効分類名称, 医薬品コード, 医薬品名, 薬価基準収載医薬品コード, 薬価, 後発品区分, 総計, year, in_hospital, oral, id_l4, Approved, KEGG, Ingredient, date, release_year, 成分, variable, id, revenue]
Index: []

[0 rows x 21 columns]


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [19]:
data["elapsed"]=data["year"]-data["release_year"]
#　不要なところを除外
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
dummy_cols=dummies.columns.to_list()

# sum by generic
sum_generic=data.groupby(["year","id","後発品区分"])[sumcolumns].sum().reset_index()

if count:
    # count ver.
    aggregated_df = pd.concat([data.groupby(['year', 'id']).first().reset_index()[['year', 'id']], 
      dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index(drop=True)
else:
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first().reset_index()[['year', 'id']],
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int)).reset_index(drop=True)
    ], axis=1).reset_index(drop=True)
# # sumをとったところを追加する

aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df.rename(columns={"総計":"sum_quantity","revenue":"sum_revenue"},inplace=True)
aggregated_df["mean_price"]=aggregated_df["sum_revenue"]/aggregated_df["sum_quantity"]
aggregated_df["lag_mean_price"]=aggregated_df.groupby(["id"])["mean_price"].shift(1)
# lag_sum_quantity
aggregated_df["lag_sum_quantity"]=aggregated_df.groupby(["id"])["sum_quantity"].shift(1)
aggregated_df.shape

(256, 45)

In [20]:
sum_generic["後発品区分"]=sum_generic["後発品区分"].astype("int")
sum_generic=sum_generic.pivot_table(index=["year","id"],columns="後発品区分",values=sumcolumns,aggfunc="sum").reset_index()
sum_generic.columns = [f"{col[0]}{col[1]}" for col in sum_generic.columns]


In [21]:
aggregated_df[sum_generic.columns]=sum_generic[sum_generic.columns]

In [22]:
sums

,year,id,総計,revenue
0,2014,1147.0,1.652349e+08,4.370214e+09
1,2014,1149.0,3.264717e+09,1.050980e+11
2,2014,1219.0,4.840017e+07,3.186312e+08
3,2014,1231.0,1.596449e+08,2.033818e+09
4,2014,1319.0,8.145575e+08,1.297159e+11
...,...,...,...,...
251,2021,4413.0,1.614168e+08,1.197058e+09
252,2021,4490.0,4.594037e+09,1.767747e+11
253,2021,6250.0,1.438952e+08,1.508488e+11
254,2021,6290.0,1.627067e+08,4.950757e+10


In [23]:
data.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital', 'oral', 'id_l4', 'Approved', 'KEGG',
       'Ingredient', 'date', 'release_year', '成分', 'variable', 'id', 'revenue',
       'elapsed'],
      dtype='object')

In [24]:
merged_df=pd.merge(data,aggregated_df,on=["year","id"])

In [25]:
merged_df.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital', 'oral', 'id_l4', 'Approved', 'KEGG',
       'Ingredient', 'date', 'release_year', '成分', 'variable', 'id', 'revenue',
       'elapsed', 'elapsed_m6', 'elapsed_m5', 'elapsed_m4', 'elapsed_m3',
       'elapsed_m2', 'elapsed_m1', 'elapsed_0', 'elapsed_1', 'elapsed_2',
       'elapsed_3', 'elapsed_4', 'elapsed_5', 'elapsed_6', 'elapsed_7',
       'elapsed_8', 'elapsed_9', 'elapsed_10', 'elapsed_11', 'elapsed_12',
       'elapsed_13', 'elapsed_14', 'elapsed_15', 'elapsed_16', 'elapsed_17',
       'elapsed_18', 'elapsed_19', 'elapsed_20', 'elapsed_21', 'elapsed_22',
       'elapsed_23', 'elapsed_24', 'elapsed_25', 'elapsed_26', 'elapsed_27',
       'elapsed_28', 'elapsed_29', 'elapsed_30', 'elapsed_31', 'sum_quantity',
       'sum_revenue', 'mean_price', 'lag_mean_price', 'lag_sum_quantity',
       'revenue0', 'revenue1', '総計0', '総計1'],
      dtype='object')

connect generic usage

In [26]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [27]:
merged_df=merged_df.merge(dusage,left_on="year",right_on="Year")
merged_df.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,sum_revenue,mean_price,lag_mean_price,lag_sum_quantity,revenue0,revenue1,総計0,総計1,Year,Percentage
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,4.370214e+09,26.448498,NaN,NaN,4.110713e+09,2.595008e+08,1.525181e+08,12716714.7,2014,51.55
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,4.370214e+09,26.448498,NaN,NaN,4.110713e+09,2.595008e+08,1.525181e+08,12716714.7,2014,51.55
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,4.370214e+09,26.448498,NaN,NaN,4.110713e+09,2.595008e+08,1.525181e+08,12716714.7,2014,51.55
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,4.370214e+09,26.448498,NaN,NaN,4.110713e+09,2.595008e+08,1.525181e+08,12716714.7,2014,51.55
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,4.370214e+09,26.448498,NaN,NaN,4.110713e+09,2.595008e+08,1.525181e+08,12716714.7,2014,51.55


In [28]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies["q_share"]=df_with_dummies["総計"]/df_with_dummies["sum_quantity"]
df_with_dummies["r_share"]=df_with_dummies["revenue"]/df_with_dummies["sum_revenue"]
# aggregated generic share in market i (id len 4)
df_with_dummies["generic_share_q"]=df_with_dummies["総計1"]/df_with_dummies["sum_quantity"]
df_with_dummies["generic_share_r"]=df_with_dummies["revenue1"]/df_with_dummies["sum_revenue"]

df_with_dummies.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0,0,0.150667,0.074626,0.076961,0.059379
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0,0,0.062008,0.054392,0.076961,0.059379
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0,0,0.304168,0.150655,0.076961,0.059379
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0,0,0.166002,0.145613,0.076961,0.059379
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0,0,0.032312,0.077089,0.076961,0.059379


In [29]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    # dummy ver
    df_with_dummies.to_csv("../merged/len4_ndb_agg_blp_DropNever.csv",encoding="shift-jis")
else:
    # count ver
    df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_blp_DropNever.csv",encoding="shift-jis")

In [30]:
df_with_dummies.shape

(25798, 107)